In [ ]:
%pip install /lakehouse/default/Files/dtb_samples-0.1-py3-none-any.whl --q

In [ ]:
dtb_item_name = ""  # TODO Update your DTB item name here
kql_db_name=""  # TODO Upate your kql db name here

In [ ]:
import sempy.fabric as fabric
import base64
import time
import uuid
import json
import com.microsoft.spark.fabric
from com.microsoft.spark.fabric.Constants import Constants
from alpinelakes.client import AlpineLakesClient, ClientConfig
from alpinelakes.utilities import AlpineLakesFabricHelper

workspace_name = fabric.resolve_workspace_name()
dtb_client = AlpineLakesClient(ClientConfig(dtb_item_name,workspace_name))
df_reader = spark.read.option(Constants.DatabaseName, dtb_client.get_database_name())
fabric_helper = AlpineLakesFabricHelper(dtb_client)
kql_script = fabric_helper.generate_eventhouse_projection(spark, df_reader)

In [ ]:
client = fabric.FabricRestClient()
workspaceId = fabric.get_workspace_id()

queryURI = ""
url = f"v1/workspaces/{workspaceId}/kqlDatabases"
response=client.get(url)
for item in response.json()['value']:
    if item['displayName'] == f"{kql_db_name}":
        queryURI = item["properties"]["queryServiceUri"]

token_string = mssparkutils.credentials.getToken(f"{queryURI}")
header = {'Content-Type':'application/json','Authorization': f'Bearer {token_string}'}
mgmturl = f"{queryURI}/v1/rest/mgmt"
payload = {
    "db": kql_db_name,
    "csl": kql_script.encode('utf-8')
}
response=client.post(mgmturl,json=payload,headers=header)
print("Successfully created Eventhouse domain projection functions.")